In [ ]:
import sys; sys.path.append('..')
import demosaicing as dm
import importlib; importlib.reload(dm)

import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,20)

In [ ]:
kodak_dir = "../data/kodak/"
kodak = dm.kodak_dataset(kodak_dir)
crop = slice(500,600),slice(300,400)
dm.show_images([kodak[18], kodak[18][crop]], layouts='12', bgr2rgb=True)

## Validation over Kodak dataset

In [ ]:
demosaic_algos = [
    ('opencv_bilinear',     lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR)),
    ('opencv_vng',          lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR_VNG)),
    ('opencv_edge_aware',   lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR_EA)),
    ('bilinear',            dm.demosaic_bilinear),
    ('smooth_hue',          dm.demosaic_smooth_hue),
    ('median_filter',       dm.demosaic_median_filter),   
    ('laplacian_corrected', dm.demosaic_laplacian_corrected)
]

history = {}
print(f"{'method':>20}\t{'mean_pnsr':>20}\t{'median_psnr':>20}")
for algo, f in demosaic_algos:
    history[algo] = dm.validate_imgs(f, kodak)
    print(f"{algo:>20}\t{np.mean(history[algo]['psnr']):>20.3f}\t{np.median(history[algo]['psnr']):>20.3f}")

In [ ]:
img_number = [18,19,22]
print(f"{'method':>20}"+"".join([f"{i:>20}" for i in img_number]))
for algo,hist in history.items():
    d = ""
    for i in img_number:
        d = d + f"{np.mean(hist['psnr'][i]):>20.3f}"
    print(f"{algo:>20}\t"+d)

## Robustnes of deomosaicing methods over additive gaussian noise

+ Takeaway
    + demosaicing methods deteriorates quickly with increasing noise levels
    + methods compared are similarly performant at high $\sigma$s',
    + 39dB ($\sigma$=1) -> 37 ($\sigma$=12) -> 34 dB ($\sigma$=20)

In [ ]:
img = kodak[18]

sigmas = [1,5,10,20]
imgs = [img]
desc = ['original']
for i, sigma in enumerate(sigmas):
    img_gn = dm.additive_gaussian_noise(img,variance=sigma**2)
    imgs.append(img_gn)
    desc.append(f'$\sigma = {sigma}$')

crops = [img[(*crop,)] for img in imgs]
dm.show_images(crops, desc, layouts='15', bgr2rgb=True)

In [ ]:
demosaic_algos = [
    ('opencv_bilinear',     lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR)),
    ('opencv_vng',          lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR_VNG)),
    ('opencv_edge_aware',   lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR_EA)),
    ('bilinear',            dm.demosaic_bilinear),
    ('smooth_hue',          dm.demosaic_smooth_hue),
    ('median_filter',       dm.demosaic_median_filter),   
    ('laplacian_corrected', dm.demosaic_laplacian_corrected)
]

sigmas = [1,5,10,20]
history = {sigma: {} for sigma in sigmas}
print(f"{'method':>20}\t{'mean_pnsr':>20}\t{'median_psnr':>20}")
for sigma in sigmas:
    print(f"sigma={sigma}:\n")
    dataset = list(map(lambda img: dm.additive_gaussian_noise(img, variance=sigma**2), kodak))
    for algo, f in demosaic_algos:
        history[sigma][algo] = dm.validate_imgs(f, dataset)
        print(f"{algo:>20}\t{np.mean(history[sigma][algo]['psnr']):>20.3f}\t{np.median(history[sigma][algo]['psnr']):>20.3f}")